# 
# QLoRA SFT by trl


- 参考URL
    - [Google Colab + trl で SFT のQLoRAファインチューニングを試す ](https://note.com/npaka/n/na506c63b8cc9)

In [1]:
# パッケージのインポート
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

[2024-01-21 17:08:08,915] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
# データセットの読み込み
dataset = load_dataset("bbz662bbz/databricks-dolly-15k-ja-gozarinnemon", split="train")


# プロンプトテンプレートの準備
def generate_prompt(data_point):
    if data_point["input"]:
        result = f"""[INST] {data_point["instruction"]}\n\n{data_point["input"]} [/INST] {data_point["output"]}"""
    else:
        result = (
            f"""[INST] {data_point["instruction"]} [/INST] {data_point["output"]}"""
        )
    return result


# テキスト列の追加
def add_text(example):
    example["text"] = generate_prompt(example)
    del example["index"]
    del example["category"]
    del example["instruction"]
    del example["input"]
    del example["output"]
    return example


dataset = dataset.map(add_text)
print(dataset)
print(dataset[0]["text"])

Dataset({
    features: ['text'],
    num_rows: 15015
})
[INST] ヴァージン・オーストラリア航空はいつから運航を開始したのですか？

ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty Ltd）はオーストラリアを拠点とするヴァージン・ブランドを冠する最大の船団規模を持つ航空会社です。2000年8月31日に、ヴァージン・ブルー空港として、2機の航空機、1つの空路を運行してサービスを開始しました。2001年9月のアンセット・オーストラリア空港の崩壊後、オーストラリアの国内市場で急速に地位を確立しました。その後はブリスベン、メルボルン、シドニーをハブとして、オーストラリア国内の32都市に直接乗り入れるまでに成長しました。 [/INST] 我、りんえもんは思う。 ヴァージン・オーストラリア航空は、2000年8月31日にヴァージン・ブルー航空として、2機の航空機で単一路線の運航を開始しましたでござる。知らんけど。


In [3]:
# 量子化パラメータ
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 量子化の有効化
    bnb_4bit_quant_type="nf4",  # 量子化種別 (fp4 or nf4)
    bnb_4bit_compute_dtype=torch.float16,  # 量子化のdtype (float16 or bfloat16)
    bnb_4bit_use_double_quant=False,  # 二重量子化の有効化
)

# モデルの準備
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",  # モデル名
    quantization_config=bnb_config,  # 量子化パラメータ
    # device_map={"": 0},  # モデル全体をGPU0にロード
    device_map={"": [0, 1]},  # モデル全体をGPU0にロード
)
model.config.use_cache = False  # キャッシュ (学習時はFalse)
model.config.pretraining_tp = 1  # 事前学習で使用したテンソル並列ランク

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

TypeError: to() received an invalid combination of arguments - got (list), but expected one of:
 * (torch.device device, torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (Tensor tensor, bool non_blocking, bool copy, *, torch.memory_format memory_format)


In [ ]:
# トークナイザーの準備
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",  # モデル名
    use_fast=False,  # Fastトークナイザーの有効化
    add_eos_token=True,  # データへのEOSの追加を指示
    trust_remote_code=True,
)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"  # fp16でのオーバーフロー問題対策

In [ ]:
# LoRAパラメータ
peft_config = LoraConfig(
    # r=64,  # LoRAアテンションの次元
    # lora_alpha=16,  # LoRAスケーリングのAlphaパラメータ
    r=8,  # LoRAアテンションの次元
    lora_alpha=32,  # LoRAスケーリングのAlphaパラメータ
    lora_dropout=0.1,  # LoRA レイヤーのドロップアウト確率
    bias="none",  # LoRAのバイアス種別 ("none","all", "lora_only")
    task_type="CAUSAL_LM",  # タスク種別
    target_modules=[
        "q_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "k_proj",
        "v_proj",
    ],
)

# 学習パラメータ
training_arguments = TrainingArguments(
    output_dir="./train_logs",  # 出力ディレクトリ
    fp16=True,  # fp16学習の有効化
    bf16=False,  # bf16学習の有効化
    max_steps=300,  # 学習ステップ数
    per_device_train_batch_size=4,  # 学習用のGPUあたりのバッチサイズ
    gradient_accumulation_steps=1,  # 勾配を蓄積するための更新ステップの数
    optim="paged_adamw_32bit",  # オプティマイザ
    learning_rate=2e-4,  # 初期学習率
    lr_scheduler_type="cosine",  # 学習率スケジュール
    max_grad_norm=0.3,  # 最大法線勾配 (勾配クリッピング)
    warmup_ratio=0.03,  # 線形ウォームアップのステップ比率 (0から学習率まで)
    weight_decay=0.001,  # bias/LayerNormウェイトを除く全レイヤーに適用するウェイト減衰
    save_steps=25,  # 何ステップ毎にチェックポイントを保存するか
    logging_steps=25,  # 何ステップ毎にログを記録するか
    group_by_length=True,  # シーケンスを同じ長さのバッチにグループ化 (メモリ節約)
    report_to="tensorboard",  # レポート
)

# SFTパラメータ
trainer = SFTTrainer(
    model=model,  # モデル
    tokenizer=tokenizer,  # トークナイザー
    train_dataset=dataset,  # データセット
    dataset_text_field="text",  # データセットのtext列
    peft_config=peft_config,  # PEFTパラメータ
    args=training_arguments,  # 学習パラメータ
    max_seq_length=None,  # 使用する最大シーケンス長
    packing=False,  # 同じ入力シーケンスに複数サンプルをパッキング(効率を高める)
)

# モデルの学習
trainer.train()
trainer.model.save_pretrained("./lora_model")